In [1]:
%load_ext autoreload
%autoreload 1

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
  
import tensorflow as tf

try:
  import tensorflow.compat.v2 as tf
except Exception:
  pass

tf.enable_v2_behavior()

print(tf.__version__)
#Connect to our own Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

#Go on the DL project folder
%cd "/content/gdrive/My Drive/NLP_Project/"
!ls


2.2.0-rc3
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/NLP_Project
cleaning.py		  model30_relu_epoch_12.h5  model30_relu_epoch_6.h5
data			  model30_relu_epoch_1.h5   model30_relu_epoch_7.h5
embedding		  model30_relu_epoch_2.h5   model30_relu_epoch_8.h5
MaLSTM.ipynb		  model30_relu_epoch_3.h5   model30_relu_epoch_9.h5
model30_relu_epoch_10.h5  model30_relu_epoch_4.h5   __pycache__
model30_relu_epoch_11.h5  model30_relu_epoch_5.h5   utils_cleaning.py


In [21]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
import re
from string import punctuation
import matplotlib.pyplot as plt
from tqdm import tqdm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import utils_cleaning as cl

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
df_train = pd.read_csv("data/train.csv")

In [6]:
train_question1 = []
cl.process_questions(train_question1, df_train.question1, 'train_question1', df_train)

train_question2 = []
cl.process_questions(train_question2, df_train.question2, 'train_question2', df_train)


train_question1 is 24.7% complete.
train_question1 is 49.5% complete.
train_question1 is 74.2% complete.
train_question1 is 98.9% complete.
train_question2 is 24.7% complete.
train_question2 is 49.5% complete.
train_question2 is 74.2% complete.
train_question2 is 98.9% complete.


In [0]:
!ls

cleaning.py  data  embedding  MaLSTM.ipynb  __pycache__  utils_cleaning.py


In [7]:
#Load word2vec embeddings
from gensim.models import KeyedVectors
fname = 'embedding/GoogleNews-vectors-negative300.bin'

#saved in a binary format to save space
w2v_model = KeyedVectors.load_word2vec_format(fname, binary = True)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
#{'a word' : vector of 300 } example : {'hello' : [0.034, ... , -0.045],...,'raised' : [0.05, 0.004,..., -0.22]}
w2v = dict(zip(w2v_model.wv.index2word, w2v_model.wv.syn0))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [0]:
max_seq_length = 50

vocabulary = dict()
inv_voc = ['<unk>']  # '<unk>' acts as a placeholder for the zero vector embedding

qs = pd.DataFrame({'q1': train_question1, 'q2': train_question2})
qs_cols = ['q1', 'q2']

In [24]:
# Iterate through the text of both questions of each pair
for index, row in tqdm(qs.iterrows()):
    for qst in qs_cols:
        q2n = []  # q2n -> numerical vector representation of each question
        for word in row[qst]:
            # Check for stopwords who do not have a word2vec mapping and ignore them
            if word not in w2v_model.vocab:
                continue

            if word not in vocabulary:
                vocabulary[word] = len(inv_voc)
                q2n.append(len(inv_voc))
                inv_voc.append(word)
            else:
                q2n.append(vocabulary[word])

        # Replace questions with equivalent numerical vector/ word-indices
        qs.at[index, qst]=q2n
    

# Prepare embedding layer
embedding_dim = 300
embeddings = np.random.randn(len(vocabulary)+1, embedding_dim) # Embedding matrix
embeddings[0] = 0 # This is to ignore the zero padding at the beginning of the sequence


404290it [01:34, 4268.12it/s]


In [26]:
# Build the embedding matrix
for word, index in tqdm(vocabulary.items()):
    if word in w2v_model.vocab:
        embeddings[index] = w2v[word]


del w2v_model, w2v

100%|██████████| 35/35 [00:00<00:00, 18170.64it/s]


In [27]:
import itertools
              
# Train-validation split
X = qs[qs_cols]
y = df_train['is_duplicate']

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.15, random_state = 102)

# Preparing train and validation feature-sets
X_train = {'left': X_train.q1, 'right': X_train.q2}
X_val = {'left': X_val.q1, 'right': X_val.q2}

# Preparing target labels
y_train = y_train.values
y_val = y_val.values


## Importing required keras libraries
import keras
from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import LSTM, Embedding, Input, Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K


# Truncate and pad input sequences
for dataset, side in itertools.product([X_train, X_val], ['left', 'right']):
    dataset[side] = sequence.pad_sequences(dataset[side], maxlen=max_seq_length)

Using TensorFlow backend.


In [62]:
def exponent_neg_manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

## Build the model
# Model variables
n_hidden = 30
batch_size = 64

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], 
                            input_length=max_seq_length, trainable=False)

left_input = Input(shape=(max_seq_length,), dtype='int32', name = 'input_1')
embed_left = embedding_layer(left_input)

right_input = Input(shape=(max_seq_length,), dtype='int32', name = 'input_2')
embed_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
lstm_layer  = LSTM(n_hidden, activation = 'relu', name = 'lstm_1_2')

left_output = lstm_layer(embed_left)
right_output = lstm_layer(embed_right)

# Manhattan distance
custom_dist = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]), 
                        output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Combine all of the above in a Model
model = Model([left_input, right_input], [custom_dist])

model.compile(loss='mean_squared_error', optimizer= 'adam', metrics=['accuracy'])

# Start training
model_save = ModelCheckpoint("model_w2c_5.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
earlyStopping = EarlyStopping(monitor="val_accuracy",patience=2)

callbacks_list = [model_save,earlyStopping]
history = model2.fit([X_train['left'], X_train['right']], y_train, batch_size=batch_size, epochs=7,
                            validation_data=([X_val['left'], X_val['right']], y_val),callbacks=callbacks_list)


"""
# Load saved weights and then compile to make it ready for further training or predictions
model.load_weights("model30_relu_epoch_3.h5", by_name = True)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

"""

Train on 343646 samples, validate on 60644 samples
Epoch 1/5
343646/343646 [==============================] - 1346s 4ms/step - loss: 0.1980 - accuracy: 0.6973 - val_loss: 0.1903 - val_accuracy: 0.7100

Epoch 00001: val_accuracy improved from -inf to 0.71005, saving model to model_w2c_5.h5
Epoch 2/5
343646/343646 [==============================] - 1322s 4ms/step - loss: 0.1882 - accuracy: 0.7137 - val_loss: 0.1850 - val_accuracy: 0.7196

Epoch 00002: val_accuracy improved from 0.71005 to 0.71961, saving model to model_w2c_5.h5
Epoch 3/5
343646/343646 [==============================] - 1315s 4ms/step - loss: 0.1848 - accuracy: 0.7216 - val_loss: 0.1834 - val_accuracy: 0.7231

Epoch 00003: val_accuracy improved from 0.71961 to 0.72311, saving model to model_w2c_5.h5
Epoch 4/5
343646/343646 [==============================] - 1309s 4ms/step - loss: 0.1826 - accuracy: 0.7254 - val_loss: 0.1828 - val_accuracy: 0.7237

Epoch 00004: val_accuracy improved from 0.72311 to 0.72370, saving model to

'\n# Load saved weights and then compile to make it ready for further training or predictions\nmodel.load_weights("model30_relu_epoch_3.h5", by_name = True)\nmodel.compile(loss=\'mean_squared_error\', optimizer=\'adam\', metrics=[\'accuracy\'])\n\n## Making predictions on validation set\npreds = model.predict([X_val[\'left\'], X_val[\'right\']])\ny_pred = [1 if x > 0.5 else 0 for x in preds] \naccuracy = (y_pred == labels).sum()/len(predictions)\n\n'

In [0]:
preds = model.predict([X_val['left'], X_val['right']])
y_pred = [1 if x > 0.5 else 0 for x in preds] 
accuracy = (y_pred == y_val).sum()/len(y_val)

In [43]:
for i in np.arange(0.1,0.9,0.1):
  y_pred = [1 if x > i else 0 for x in preds] 
  accuracy = (y_pred == y_val).sum()/len(y_val)
  print(accuracy)


thresholds = []
for thresh in np.arange(0.1, 0.95, 0.01):
    thresh = np.round(thresh, 3)
    res = metrics.f1_score(val_y, (pred_val_y > thresh).astype(int))
    thresholds.append([thresh, res])
    print("F1 score at threshold {0} is {1}".format(thresh, res))
    
thresholds.sort(key=lambda x: x[1], reverse=True)
best_thresh = thresholds[0][0]
print("Best threshold: ", best_thresh)

0.4879130664204208
0.6204900732141679
0.7057087263373129
0.7377481696458017
0.7377811490007256
0.7194446276630829
0.6969527076050392
0.6759613481960293


In [0]:
y_pred = [1 if x > 0.5 else 0 for x in preds] 
TP = (y_pred == y_val).sum()/len(y_val)
print(accuracy)

In [59]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred)
tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
recall = tp/(tp+fn)
precision = tp/(tp+fp)
f1_score = 2 * (precision * recall)/(precision+ recall)

print("Accuracy :",accuracy)
print("recall :",recall)
print("precision :",precision)
print("f1_Score :",f1_score)


Accuracy : 0.7377811490007256
recall : 0.5068584960457531
precision : 0.6998581035227343
f1_Score : 0.587924332728686


In [36]:
sum(y_val)/len(y_val)

0.36905547127498184